# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/deepseek-ai/DeepSeek-V3',
 '/Qwen/QVQ-72B-Preview',
 '/answerdotai/ModernBERT-base',
 '/IamCreateAI/Ruyi-Mini-7B',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/Qwen/QVQ-72B-preview',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/amphora/QwQ-LongCoT-130K',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/dif

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'Company page', 'url': 'https://huggingface.co/'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co'},
  {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Support and Resources', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status updates', 'url': 'https://status.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        try:
            result += Website(link["url"]).get_contents()
        except:
            result += '\nFaulty site!\n'
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'Company page', 'url': 'https://huggingface.co'}, {'type': 'About/Having a Conversation: Building and Improving Hugging Face', 'url': 'https://blog.huggingface.co/beginning-with-hugging-face-7d98bcbab8f6'}, {'type': 'Jobs/Careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'About/Holding a Conference', 'url': 'https://blog.huggingface.co/holding-a-conference-ebea1bf66e7c'}, {'type': 'About/Our History,', 'url': 'https://about.huggingface.co/history/hugging-face-the-organization-evolves'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-V3-Base
Updated
3 days ago
•
3.37k
•
837
deepseek-ai/DeepSeek-V3
Updated
3 day

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/brand'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n3 days ago\n•\n3.37k\n•\n837\ndeepseek-ai/DeepSeek-V3\nUpdated\n3 days ago\n•\n11.9k\n•\n561\nQwen/QVQ-72B-Preview\nUpdated\n5 days ago\n•\n22k\n•\n369\nanswerdotai/ModernBERT-base\nUpdated\n4 days ago\n•\n44.2k\n•\n514\nIamCreateAI/Ruyi-Mini-7B\nUpdated\n4 days ago\n•\n14.1k\n•\n532\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n2.26k\n🏢\nTRELLIS\nScalable and Versatile 3D Generation from im

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'careers page', 'url': 'https://apply.workable.com/huggingfaces'}, {'type': 'About page', 'url': 'https://huggingface.co/brand'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Contact us page', 'url': 'https://ui.endpoints.huggingface.co/login'}, {'type': 'Jobs/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face: Revolutionizing Machine Learning Collaboration
=====================================

Welcome to Hugging Face, a platform that empowers the machine learning community to collaborate, share, and experiment with open-source models and tools.

## Our Mission
----------------

We are building the foundation of ML tooling to enable an open and ethical AI future. Our mission is to provide a collaborative ecosystem for machine learning engineers, scientists, and end-users to learn, share, and build together.

## What We Offer
-----------------

*   **Hub**: A central place where anyone can share, explore, discover, and experiment with open-source ML models and tools.
*   **Models and Datasets**: Access over 400k+ models, datasets, and applications for computer vision, audio, NLP, and more.
*   **Spaces**: Scalable and versatile 3D generation from images, QVQ 72B Preview, Kolors Virtual Try-On, and other innovative tools.
*   **Enterprise Solutions**: Advanced platform for building AI with enterprise-grade security, access controls, and dedicated support.

## Our Community
-----------------

Join our community of over 50,000 organizations and more than 1 million followers across top tech companies like Meta, Google, Amazon Web Services, Intel, Microsoft, and Grammarly. Some notable members include:

*   AI2 (non-profit)
*   Enterprise company at Meta
*   Company at Amazon Web Services
*   Company at Google
*   Company at Intel
*   Company at Microsoft
*   Writer (Enterprise company)
*   Writer (Company)

## Our Tools and Technologies
-------------------------------

*   **Transformers**: State-of-the-art ML for Pytorch, TensorFlow, and JAX.
*   **Diffusers**: State-of-the-art diffusion models for image and audio generation in PyTorch.
*   **Safetensors**: Simple, safe way to store and distribute neural networks weights safely and quickly.
*   **Hub Python Library**: Client library for managing repositories from your Python runtime.
*   **Tokenizers**: Fast tokenizers optimized for both research and production.

## Get Started
----------------

Sign up for our platform today and join the machine learning community that is shaping the future of AI. With easy access to models, datasets, and tools, you can accelerate your ML projects and build your portfolio.

### Sign Up | [Learn More About Our Pricing Plans](/pricing)

### Explore Our Open Source Stack

*   **Documentation**: Learn more about our technology and platform.
*   **Blog**: Stay up-to-date with the latest news, insights, and best practices in machine learning.

[Image: Hugging Face logo]

Join us on:

*   [GitHub](https://github.com/huggingface)
*   [Twitter](https://twitter.com/huggingface)
*   [LinkedIn](https://linkedin.com/company/14484642)
*   [Discord](https://discord.com/invite/huggingface)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co'}, {'type': 'Blog', 'url': 'https://blog.huggingface.co'}, {'type': 'Discussions forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'Status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter handle', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Join/Sign up page', 'url': 'https://apply.workable.com/huggingface/', 'altUrl': '/join'}, {'type': 'Careers/Jobs page', 'url': 'https://huggingface.co/careers'}]}


# Hugging Face Brochure

[Image of a person hugging a robot, with a cityscape in the background]

Hugging Face: Building the Future of AI Together

At Hugging Face, we're dedicated to creating a platform where the machine learning community can collaborate on models, datasets, and applications. Our mission is to accelerate scientific progress and innovation in the field of artificial intelligence.

## Our Story

From its inception, Hugging Face has been driven by a passion for building an open-source foundation for machine learning tooling with the community. We've established ourselves as leaders in the field, providing state-of-the-art solutions for research and production.

## Our Key Offerings

*   **Models**: Browse over 400,000+ models on our platform, serving text, image, video, audio, or even 3D.
*   **Datasets**: Access over 100,000+ datasets for your NLP tasks in computer vision, audio, and more.
*   **Spaces**: Collaborate with others on unlimited public models, datasets, and applications.

## What Makes Us Unique

*   **Community-Driven**: Our platform is built by the machine learning community, for the machine learning community.
*   **Agility**: We offer paid Compute and Enterprise solutions to help accelerate your ML projects.
*   **Speed**: Easily train and use PyTorch models with multi-GPU, TPU, mixed-precision.

## Our Users

Hugging Face is used by a diverse range of organizations and individuals, including:

| Company            | Models   | Followers |
 |:------------------:|---------:|----------:|
| AI2                | 366      |    1.79k |
| Meta               |     2.06k |    3.92k  |
| Amazon Web Services  |       21  |   2.47k |
| Google              |        911|    6.01k |

## Join the Hugging Face Community

Ready to be part of a vibrant community that's shaping the future of AI? Explore our resources, documentation, and job openings today.

[Image of a person in front of a city skyline, with a laptop and pen]

Hugging Face - Building the Future of AI Together

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>